# Big Five Personality Recognition using LUKE on RealPersonaChat

**プロジェクト概要:**
- データセット: RealPersonaChat (14,000対話、233話者)
- タスク: Big Five性格特性の回帰予測
- モデル: LUKE (studio-ousia/luke-japanese-base)
- 設定: モノローグ（話者の発話のみ）

**評価指標:**
- 回帰: MAE, RMSE, Pearson, Spearman相関
- 分類: Accuracy, Balanced Accuracy, Precision, Recall, F1

**メモリ最適化 (T4 GPU対応):**
- バッチサイズ: 4
- Max Length: 256 tokens
- 勾配蓄積: 8ステップ（実質バッチサイズ32相当）
- 勾配チェックポイント: 有効化
- Mixed Precision (FP16): 有効化

**推奨環境:**
- Google Colab with T4 GPU (15GB VRAM)
- ランタイム再起動後に実行することを推奨

# Big Five Personality Recognition using LUKE on RealPersonaChat

**プロジェクト概要:**
- データセット: RealPersonaChat (14,000対話、233話者)
- タスク: Big Five性格特性の回帰予測
- モデル: LUKE (studio-ousia/luke-japanese-base)
- 設定: モノローグ（話者の発話のみ）

**評価指標:**
- 回帰: MAE, RMSE, Pearson, Spearman相関
- 分類: Accuracy, Balanced Accuracy, Precision, Recall, F1

## 1. 環境セットアップ

In [1]:
# ライブラリインストール
!pip install -q transformers datasets torch scikit-learn pandas numpy matplotlib seaborn

In [2]:
# インポート
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW  # transformersではなくtorch.optimからインポート
from transformers import (
    AutoTokenizer,  # LukeTokenizerの代わりにAutoTokenizerを使用
    LukeModel,
    get_linear_schedule_with_warmup
)
import json
import requests
from pathlib import Path
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    mean_absolute_error,
    mean_squared_error,
    accuracy_score,
    balanced_accuracy_score,
    precision_recall_fscore_support
)
from scipy.stats import pearsonr, spearmanr
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm
import warnings
warnings.filterwarnings('ignore')

# デバイス設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

Using device: cuda
GPU: Tesla T4
VRAM: 15.83 GB


## 2. データ読み込み

RealPersonaChatデータセットをGitHubから直接読み込みます。
- 話者データ: `interlocutors.json` (233名)
- 対話データ: `dialogues/*.json` (最大14,000件)

**注意**: 全14,000件のダウンロードには時間がかかります。
- テスト用: `num_dialogues = 1000` (約2-3分)
- 本番用: `num_dialogues = 14000` (約30-40分)

**Google Driveキャッシュ:**
- 初回: ダウンロード + 保存（2-3分）
- 2回目以降: キャッシュから読み込み（5秒）

# データセット読み込み（GitHubから直接ダウンロード + Google Drive キャッシュ）
import json
import requests
from pathlib import Path
import pickle

print("RealPersonaChatデータセット読み込み中...")

# Google Drive上のキャッシュディレクトリ
CACHE_DIR = Path("/content/drive/MyDrive/real_persona_chat_cache")
CACHE_DIR.mkdir(exist_ok=True)

INTERLOCUTORS_CACHE = CACHE_DIR / "interlocutors.pkl"
DIALOGUES_CACHE = CACHE_DIR / "dialogues_1000.pkl"

# GitHubのrawファイルURL
BASE_URL = "https://raw.githubusercontent.com/nu-dialogue/real-persona-chat/main/real_persona_chat"

# 1. Interlocutors (話者データ) を読み込み
print("\n" + "="*70)
print("話者データを読み込み中...")
print("="*70)

if INTERLOCUTORS_CACHE.exists():
    print("✅ キャッシュから読み込み（高速）")
    with open(INTERLOCUTORS_CACHE, 'rb') as f:
        interlocutor_dict = pickle.load(f)
    print(f"✅ 話者データ読み込み完了: {len(interlocutor_dict)} speakers")
else:
    print("📥 GitHubからダウンロード中...")
    interlocutors_url = f"{BASE_URL}/interlocutors.json"
    response = requests.get(interlocutors_url)
    interlocutors_raw = response.json()
    
    # データ構造を変換
    if isinstance(interlocutors_raw, dict):
        interlocutor_dict = interlocutors_raw
    elif isinstance(interlocutors_raw, list):
        interlocutor_dict = {
            item['interlocutor_id']: item
            for item in interlocutors_raw
        }
    else:
        print(f"⚠️  予期しないデータ型: {type(interlocutors_raw)}")
        interlocutor_dict = {}
    
    # キャッシュに保存
    with open(INTERLOCUTORS_CACHE, 'wb') as f:
        pickle.dump(interlocutor_dict, f)
    print(f"✅ 話者データ保存完了: {INTERLOCUTORS_CACHE}")
    print(f"✅ 話者データ読み込み完了: {len(interlocutor_dict)} speakers")

# サンプルデータを表示
if interlocutor_dict:
    first_speaker_id = list(interlocutor_dict.keys())[0]
    sample_speaker = interlocutor_dict[first_speaker_id]
    print(f"\n📊 話者データサンプル（話者ID: {first_speaker_id}）:")
    print(f"  利用可能なキー: {list(sample_speaker.keys())}")
    if 'personality' in sample_speaker:
        print(f"  Personality keys: {list(sample_speaker['personality'].keys())[:10]}")

# 2. Dialogue データを読み込み
print("\n" + "="*70)
print("対話データを読み込み中...")
print("="*70)

num_dialogues = 1000  # 必要に応じて変更（最大14000）

if DIALOGUES_CACHE.exists():
    print(f"✅ キャッシュから読み込み（高速） - {num_dialogues}件")
    with open(DIALOGUES_CACHE, 'rb') as f:
        dialogue_data = pickle.load(f)
    print(f"✅ 対話データ読み込み完了: {len(dialogue_data)} dialogues")
else:
    print(f"📥 GitHubからダウンロード中... ({num_dialogues}件)")
    print("（注：初回のみ時間がかかります。次回以降はキャッシュから高速読み込み）")
    
    dialogue_data = []
    failed_downloads = 0
    
    for i in tqdm(range(1, num_dialogues + 1), desc="Downloading dialogues"):
        dialogue_id = f"{i:05d}"
        dialogue_url = f"{BASE_URL}/dialogues/{dialogue_id}.json"
        
        try:
            response = requests.get(dialogue_url)
            if response.status_code == 200:
                dialogue = response.json()
                dialogue_data.append(dialogue)
            else:
                failed_downloads += 1
        except Exception as e:
            failed_downloads += 1
            if i <= 10:
                print(f"Failed to download {dialogue_id}: {e}")
    
    # キャッシュに保存
    with open(DIALOGUES_CACHE, 'wb') as f:
        pickle.dump(dialogue_data, f)
    print(f"\n✅ 対話データ保存完了: {DIALOGUES_CACHE}")
    print(f"✅ 対話データ読み込み完了: {len(dialogue_data)} dialogues")
    if failed_downloads > 0:
        print(f"⚠️  ダウンロード失敗: {failed_downloads} dialogues")

# データ構造を確認
print("\n📊 対話データサンプル（最初の1件）:")
if dialogue_data:
    sample = dialogue_data[0]
    print(f"  利用可能なキー: {list(sample.keys())}")
    print(f"  Dialogue ID: {sample.get('dialogue_id', 'N/A')}")
    print(f"  Interlocutors: {sample.get('interlocutors', [])}")
    print(f"  Utterances: {len(sample.get('utterances', []))} utterances")

print("\n" + "="*70)
print("💡 次回以降はキャッシュから高速読み込みします（数秒）")
print("="*70)

In [4]:
# データセット読み込み（GitHubから直接ダウンロード）
import json
import requests
from pathlib import Path

print("GitHubからRealPersonaChatデータセット読み込み中...")

# データ保存ディレクトリ
data_dir = Path("/content/real_persona_chat_data")
data_dir.mkdir(exist_ok=True)

# GitHubのrawファイルURL
BASE_URL = "https://raw.githubusercontent.com/nu-dialogue/real-persona-chat/main/real_persona_chat"

# 1. Interlocutors (話者データ) をダウンロード
print("\n話者データをダウンロード中...")
interlocutors_url = f"{BASE_URL}/interlocutors.json"
response = requests.get(interlocutors_url)
interlocutors_raw = response.json()

# データ構造を確認
print(f"\n📊 話者データの型: {type(interlocutors_raw)}")
if isinstance(interlocutors_raw, dict):
    print(f"📊 辞書のキー: {list(interlocutors_raw.keys())[:5]}")  # 最初の5つのキーを表示
    # 辞書構造の場合、値がリストか確認
    first_key = list(interlocutors_raw.keys())[0]
    print(f"📊 最初の要素の型: {type(interlocutors_raw[first_key])}")

    # 辞書構造を変換
    if isinstance(interlocutors_raw[first_key], dict):
        # {speaker_id: {data}} 形式の場合
        interlocutor_dict = interlocutors_raw
        print(f"\n✅ 話者データ読み込み完了: {len(interlocutor_dict)} speakers")
    else:
        print(f"⚠️  予期しないデータ構造です")
        interlocutor_dict = {}
elif isinstance(interlocutors_raw, list):
    print(f"📊 リストの長さ: {len(interlocutors_raw)}")
    # リスト構造の場合、辞書に変換
    interlocutor_dict = {
        item['interlocutor_id']: item
        for item in interlocutors_raw
    }
    print(f"\n✅ 話者データ読み込み完了: {len(interlocutor_dict)} speakers")
else:
    print(f"⚠️  予期しないデータ型: {type(interlocutors_raw)}")
    interlocutor_dict = {}

# サンプルデータを表示
if interlocutor_dict:
    first_speaker_id = list(interlocutor_dict.keys())[0]
    sample_speaker = interlocutor_dict[first_speaker_id]
    print(f"\n📊 話者データサンプル（話者ID: {first_speaker_id}）:")
    print(f"  利用可能なキー: {list(sample_speaker.keys())}")
    if 'personality' in sample_speaker:
        print(f"  Personality keys: {list(sample_speaker['personality'].keys())[:10]}")  # 最初の10個

# 2. Dialogue データをダウンロード
print("\n" + "="*70)
print("対話データをダウンロード中...")
print("（注：全14,000件のダウンロードには時間がかかります）")
print("テスト用に最初の1,000件をダウンロードします...")
print("="*70 + "\n")

dialogue_data = []
num_dialogues = 1000  # 必要に応じて変更（最大14000）
failed_downloads = 0

for i in tqdm(range(1, num_dialogues + 1), desc="Downloading dialogues"):
    dialogue_id = f"{i:05d}"  # 00001形式
    dialogue_url = f"{BASE_URL}/dialogues/{dialogue_id}.json"

    try:
        response = requests.get(dialogue_url)
        if response.status_code == 200:
            dialogue = response.json()
            dialogue_data.append(dialogue)
        else:
            failed_downloads += 1
    except Exception as e:
        failed_downloads += 1
        if i <= 10:  # 最初の10件のエラーだけ表示
            print(f"Failed to download {dialogue_id}: {e}")

print(f"\n✅ 対話データ読み込み完了: {len(dialogue_data)} dialogues")
if failed_downloads > 0:
    print(f"⚠️  ダウンロード失敗: {failed_downloads} dialogues")

# データ構造を確認
print("\n📊 対話データサンプル（最初の1件）:")
if dialogue_data:
    sample = dialogue_data[0]
    print(f"  利用可能なキー: {list(sample.keys())}")
    print(f"  Dialogue ID: {sample.get('dialogue_id', 'N/A')}")
    print(f"  Interlocutors: {sample.get('interlocutors', [])}")
    print(f"  Utterances: {len(sample.get('utterances', []))} utterances")

GitHubからRealPersonaChatデータセット読み込み中...

話者データをダウンロード中...

📊 話者データの型: <class 'dict'>
📊 辞書のキー: ['AH', 'HP', 'CX', 'GS', 'CF']
📊 最初の要素の型: <class 'dict'>

✅ 話者データ読み込み完了: 233 speakers

📊 話者データサンプル（話者ID: AH）:
  利用可能なキー: ['interlocutor_id', 'persona', 'personality', 'demographic_information', 'text_chat_experience']
  Personality keys: ['BigFive_Openness', 'BigFive_Conscientiousness', 'BigFive_Extraversion', 'BigFive_Agreeableness', 'BigFive_Neuroticism', 'KiSS18_BasicSkill', 'KiSS18_AdvancedSkill', 'KiSS18_EmotionalManagementSkill', 'KiSS18_OffenceManagementSkill', 'KiSS18_StressManagementSkill']

対話データをダウンロード中...
（注：全14,000件のダウンロードには時間がかかります）
テスト用に最初の1,000件をダウンロードします...




✅ 対話データ読み込み完了: 972 dialogues
⚠️  ダウンロード失敗: 28 dialogues

📊 対話データサンプル（最初の1件）:
  利用可能なキー: ['dialogue_id', 'interlocutors', 'utterances', 'evaluations']
  Dialogue ID: 1
  Interlocutors: ['AA', 'AB']
  Utterances: 30 utterances


In [5]:
# モノローグデータの作成
# 各話者の発話のみを抽出してサンプルを作成

def create_monologue_dataset(dialogue_data, interlocutor_dict):
    """
    対話データから各話者のモノローグを抽出

    Args:
        dialogue_data: List of dialogue dictionaries
        interlocutor_dict: 話者ID -> 話者情報の辞書

    Returns:
        monologues: List[Dict]
            - 'speaker_id': str
            - 'text': str (全発話を連結)
            - 'personality': Dict[str, float] (Big Five scores)
    """
    monologues = []

    print("\nモノローグ抽出中...")

    for dialogue in tqdm(dialogue_data, desc="Processing dialogues"):
        # 各話者の発話を収集
        speaker_utterances = {}

        for utterance in dialogue.get('utterances', []):
            speaker_id = utterance.get('interlocutor_id')
            text = utterance.get('text', '')

            if not speaker_id or not text:
                continue

            if speaker_id not in speaker_utterances:
                speaker_utterances[speaker_id] = []
            speaker_utterances[speaker_id].append(text)

        # 各話者のモノローグを作成
        for speaker_id, utterances in speaker_utterances.items():
            # 話者情報を取得
            if speaker_id not in interlocutor_dict:
                continue  # 話者情報がない場合はスキップ

            speaker_info = interlocutor_dict[speaker_id]

            # Big Five スコアを抽出
            personality_data = speaker_info.get('personality', {})

            # Big Five特性のスコアを取得（1-7スケール）
            # データ構造を確認して適切なキー名を使用
            big_five_scores = {
                'Openness': personality_data.get('big_five_openness', personality_data.get('openness', 4.0)),
                'Conscientiousness': personality_data.get('big_five_conscientiousness', personality_data.get('conscientiousness', 4.0)),
                'Extraversion': personality_data.get('big_five_extraversion', personality_data.get('extraversion', 4.0)),
                'Agreeableness': personality_data.get('big_five_agreeableness', personality_data.get('agreeableness', 4.0)),
                'Neuroticism': personality_data.get('big_five_neuroticism', personality_data.get('neuroticism', 4.0)),
            }

            # スコアが取得できているか確認
            if all(score == 4.0 for score in big_five_scores.values()):
                # デフォルト値のままの場合は他のキー名を試す
                for key, value in personality_data.items():
                    if 'openness' in key.lower():
                        big_five_scores['Openness'] = value
                    elif 'conscientiousness' in key.lower():
                        big_five_scores['Conscientiousness'] = value
                    elif 'extraversion' in key.lower():
                        big_five_scores['Extraversion'] = value
                    elif 'agreeableness' in key.lower():
                        big_five_scores['Agreeableness'] = value
                    elif 'neuroticism' in key.lower():
                        big_five_scores['Neuroticism'] = value

            monologues.append({
                'speaker_id': speaker_id,
                'text': ' '.join(utterances),
                'personality': big_five_scores
            })

    return monologues

# モノローグデータ作成
monologue_data = create_monologue_dataset(dialogue_data, interlocutor_dict)

print(f"\n✅ モノローグサンプル数: {len(monologue_data)}")
if monologue_data:
    print(f"\n📊 サンプル例:")
    print(f"  話者ID: {monologue_data[0]['speaker_id']}")
    print(f"  テキスト長: {len(monologue_data[0]['text'])} 文字")
    print(f"  Big Five: {monologue_data[0]['personality']}")


モノローグ抽出中...


Processing dialogues:   0%|          | 0/972 [00:00<?, ?it/s]


✅ モノローグサンプル数: 1944

📊 サンプル例:
  話者ID: AA
  テキスト長: 248 文字
  Big Five: {'Openness': 4.25, 'Conscientiousness': 3.5, 'Extraversion': 2.9, 'Agreeableness': 5.083333333333333, 'Neuroticism': 4.416666666666667}


In [6]:
# データ分割（話者ベース）
# 論文準拠: Train/Val/Test = 8:1:1, 話者が重複しないように分割

def split_by_speaker(monologue_data, train_ratio=0.8, val_ratio=0.1, test_ratio=0.1, seed=42):
    """
    話者が重複しないようにデータ分割
    """
    # 話者IDごとにグループ化
    speaker_groups = {}
    for sample in monologue_data:
        speaker_id = sample['speaker_id']
        if speaker_id not in speaker_groups:
            speaker_groups[speaker_id] = []
        speaker_groups[speaker_id].append(sample)

    # 話者リスト
    speakers = list(speaker_groups.keys())
    np.random.seed(seed)
    np.random.shuffle(speakers)

    # 分割
    n_speakers = len(speakers)
    n_train = int(n_speakers * train_ratio)
    n_val = int(n_speakers * val_ratio)

    train_speakers = speakers[:n_train]
    val_speakers = speakers[n_train:n_train+n_val]
    test_speakers = speakers[n_train+n_val:]

    # サンプル抽出
    train_data = [s for spk in train_speakers for s in speaker_groups[spk]]
    val_data = [s for spk in val_speakers for s in speaker_groups[spk]]
    test_data = [s for spk in test_speakers for s in speaker_groups[spk]]

    print(f"\n📊 データ分割結果:")
    print(f"  Train: {len(train_speakers)} speakers, {len(train_data)} samples")
    print(f"  Val:   {len(val_speakers)} speakers, {len(val_data)} samples")
    print(f"  Test:  {len(test_speakers)} speakers, {len(test_data)} samples")

    return train_data, val_data, test_data

# データ分割実行
train_data, val_data, test_data = split_by_speaker(monologue_data)

print(f"\n✅ データ分割完了")


📊 データ分割結果:
  Train: 56 speakers, 1482 samples
  Val:   7 speakers, 143 samples
  Test:  7 speakers, 319 samples

✅ データ分割完了


## 3. データ前処理

- LUKEトークナイザーでテキストをエンコード
- Big Fiveスコアを正規化（1-7 → 0-1）

In [ ]:
# PyTorch Dataset クラス
class PersonalityDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=256):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        sample = self.data[idx]
        
        # テキストのトークナイゼーション
        encoding = self.tokenizer(
            sample['text'],
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        
        # Big Five スコアの正規化（1-7 → 0-1）
        personality_scores = torch.tensor([
            (sample['personality'][trait] - 1) / 6  # 1-7 を 0-1 に正規化
            for trait in BIG_FIVE_TRAITS
        ], dtype=torch.float32)
        
        return {
            'input_ids': encoding['input_ids'].squeeze(0),
            'attention_mask': encoding['attention_mask'].squeeze(0),
            'labels': personality_scores
        }

# データローダー作成
BATCH_SIZE = 4  # 8 → 4に変更（さらにメモリ節約）

train_dataset = PersonalityDataset(train_data, tokenizer)
val_dataset = PersonalityDataset(val_data, tokenizer)
test_dataset = PersonalityDataset(test_data, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

print(f"\n✅ Dataset準備完了")
print(f"  Max Length: 256 tokens (メモリ最適化)")
print(f"  Batch Size: {BATCH_SIZE} (極小メモリモード)")
print(f"  Train batches: {len(train_loader)}")
print(f"  Val batches: {len(val_loader)}")
print(f"  Test batches: {len(test_loader)}")

## 4. モデル構築

LUKE + 5つの回帰ヘッド（各Big Five特性に1つ）

**注意**: モデル構築前にGPUメモリをクリアします。

# GPUメモリクリーンアップ（モデル構築前）
import gc
torch.cuda.empty_cache()
gc.collect()
print("✅ GPUメモリをクリアしました")

# LUKE Personality Recognition Model
class LukePersonalityModel(nn.Module):
    def __init__(self, model_name=MODEL_NAME, num_traits=5):
        super().__init__()
        self.luke = LukeModel.from_pretrained(model_name)
        
        # 勾配チェックポイントを有効化（メモリ削減）
        self.luke.gradient_checkpointing_enable()
        
        self.hidden_size = self.luke.config.hidden_size
        
        # 各Big Five特性に対する回帰ヘッド
        self.regression_heads = nn.ModuleList([
            nn.Sequential(
                nn.Linear(self.hidden_size, 256),
                nn.ReLU(),
                nn.Dropout(0.1),
                nn.Linear(256, 1)
            )
            for _ in range(num_traits)
        ])
    
    def forward(self, input_ids, attention_mask):
        # LUKE encoding
        outputs = self.luke(
            input_ids=input_ids,
            attention_mask=attention_mask
        )
        
        # [CLS] token の hidden state
        pooled_output = outputs.last_hidden_state[:, 0, :]  # [batch_size, hidden_size]
        
        # 各特性の予測
        predictions = []
        for head in self.regression_heads:
            pred = head(pooled_output)  # [batch_size, 1]
            predictions.append(pred)
        
        # [batch_size, 5]
        predictions = torch.cat(predictions, dim=1)
        
        # 0-1 の範囲に制限（Sigmoid）
        predictions = torch.sigmoid(predictions)
        
        return predictions

# モデル初期化
print("モデル初期化中...")
model = LukePersonalityModel().to(device)

print(f"✅ モデルパラメータ数: {sum(p.numel() for p in model.parameters()) / 1e6:.2f}M")
print(f"✅ 勾配チェックポイント: 有効化（メモリ削減）")

# メモリ使用量を確認
if torch.cuda.is_available():
    allocated = torch.cuda.memory_allocated(0) / 1e9
    reserved = torch.cuda.memory_reserved(0) / 1e9
    print(f"✅ GPU Memory: {allocated:.2f}GB allocated, {reserved:.2f}GB reserved")

In [9]:
# LUKE Personality Recognition Model
class LukePersonalityModel(nn.Module):
    def __init__(self, model_name=MODEL_NAME, num_traits=5):
        super().__init__()
        self.luke = LukeModel.from_pretrained(model_name)
        self.hidden_size = self.luke.config.hidden_size

        # 各Big Five特性に対する回帰ヘッド
        self.regression_heads = nn.ModuleList([
            nn.Sequential(
                nn.Linear(self.hidden_size, 256),
                nn.ReLU(),
                nn.Dropout(0.1),
                nn.Linear(256, 1)
            )
            for _ in range(num_traits)
        ])

    def forward(self, input_ids, attention_mask):
        # LUKE encoding
        outputs = self.luke(
            input_ids=input_ids,
            attention_mask=attention_mask
        )

        # [CLS] token の hidden state
        pooled_output = outputs.last_hidden_state[:, 0, :]  # [batch_size, hidden_size]

        # 各特性の予測
        predictions = []
        for head in self.regression_heads:
            pred = head(pooled_output)  # [batch_size, 1]
            predictions.append(pred)

        # [batch_size, 5]
        predictions = torch.cat(predictions, dim=1)

        # 0-1 の範囲に制限（Sigmoid）
        predictions = torch.sigmoid(predictions)

        return predictions

# モデル初期化
model = LukePersonalityModel().to(device)
print(f"モデルパラメータ数: {sum(p.numel() for p in model.parameters()) / 1e6:.2f}M")

config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

モデルパラメータ数: 280.10M


## 5. 学習設定

In [10]:
# ハイパーパラメータ
LEARNING_RATE = 1e-5
NUM_EPOCHS = 20
WARMUP_STEPS = 150
EARLY_STOPPING_PATIENCE = 3

# Optimizer
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)

# Learning Rate Scheduler
total_steps = len(train_loader) * NUM_EPOCHS
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=WARMUP_STEPS,
    num_training_steps=total_steps
)

# 損失関数（MAE）
criterion = nn.L1Loss()  # Mean Absolute Error

print(f"\n✅ 学習設定完了")
print(f"  Learning Rate: {LEARNING_RATE}")
print(f"  Total Steps: {total_steps}")
print(f"  Warmup Steps: {WARMUP_STEPS}")


✅ 学習設定完了
  Learning Rate: 1e-05
  Total Steps: 940
  Warmup Steps: 150


In [11]:
# 評価関数
def evaluate_model(model, dataloader, device):
    model.eval()
    all_predictions = []
    all_labels = []
    total_loss = 0

    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Evaluating"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            predictions = model(input_ids, attention_mask)
            loss = criterion(predictions, labels)

            total_loss += loss.item()
            all_predictions.append(predictions.cpu().numpy())
            all_labels.append(labels.cpu().numpy())

    all_predictions = np.vstack(all_predictions)
    all_labels = np.vstack(all_labels)
    avg_loss = total_loss / len(dataloader)

    return all_predictions, all_labels, avg_loss

print("評価関数準備完了")

評価関数準備完了


# 学習履歴
history = {
    'train_loss': [],
    'val_loss': [],
    'val_mae': []
}

best_val_loss = float('inf')
patience_counter = 0

# 勾配蓄積（実質的なバッチサイズを大きくしつつメモリを節約）
ACCUMULATION_STEPS = 8  # 4 * 8 = 実質バッチサイズ32相当

# Mixed Precision (FP16) を使用してメモリ削減
from torch.cuda.amp import autocast, GradScaler
scaler = GradScaler()

print("="*70)
print("学習開始")
print(f"実質バッチサイズ: {BATCH_SIZE * ACCUMULATION_STEPS} (勾配蓄積)")
print(f"Mixed Precision: 有効化 (FP16)")
print("="*70)

# 学習ループ
for epoch in range(NUM_EPOCHS):
    print(f"\n{'='*70}")
    print(f"Epoch {epoch+1}/{NUM_EPOCHS}")
    print(f"{'='*70}")
    
    # Training
    model.train()
    train_loss = 0
    
    optimizer.zero_grad()  # 最初にゼロ化
    
    for batch_idx, batch in enumerate(tqdm(train_loader, desc="Training")):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        
        # Mixed Precision (FP16) で forward pass
        with autocast():
            predictions = model(input_ids, attention_mask)
            loss = criterion(predictions, labels)
            loss = loss / ACCUMULATION_STEPS  # 勾配蓄積のためにスケール
        
        # Backward pass with scaled gradients
        scaler.scale(loss).backward()
        
        # 勾配蓄積：ACCUMULATION_STEPS回ごとにパラメータ更新
        if (batch_idx + 1) % ACCUMULATION_STEPS == 0:
            scaler.step(optimizer)
            scaler.update()
            scheduler.step()
            optimizer.zero_grad()
        
        train_loss += loss.item() * ACCUMULATION_STEPS  # 元のスケールに戻す
        
        # メモリクリーンアップ（10バッチごと）
        if (batch_idx + 1) % 10 == 0:
            torch.cuda.empty_cache()
    
    # 残りの勾配を処理
    if (len(train_loader) % ACCUMULATION_STEPS) != 0:
        scaler.step(optimizer)
        scaler.update()
        optimizer.zero_grad()
    
    avg_train_loss = train_loss / len(train_loader)
    history['train_loss'].append(avg_train_loss)
    
    # メモリクリーンアップ
    torch.cuda.empty_cache()
    
    # Validation
    val_predictions, val_labels, val_loss = evaluate_model(model, val_loader, device)
    val_mae = mean_absolute_error(val_labels, val_predictions)
    
    history['val_loss'].append(val_loss)
    history['val_mae'].append(val_mae)
    
    print(f"\n📊 Results:")
    print(f"  Train Loss: {avg_train_loss:.4f}")
    print(f"  Val Loss: {val_loss:.4f}")
    print(f"  Val MAE: {val_mae:.4f}")
    
    # GPU メモリ使用状況
    if torch.cuda.is_available():
        allocated = torch.cuda.memory_allocated(0) / 1e9
        reserved = torch.cuda.memory_reserved(0) / 1e9
        print(f"  GPU Memory: {allocated:.2f}GB allocated, {reserved:.2f}GB reserved")
    
    # Early Stopping & Checkpoint
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
        # モデル保存
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'val_loss': val_loss,
        }, f"{CHECKPOINT_DIR}/best_model.pt")
        print("  ✅ Best model saved!")
    else:
        patience_counter += 1
        print(f"  ⚠️  Patience: {patience_counter}/{EARLY_STOPPING_PATIENCE}")
        if patience_counter >= EARLY_STOPPING_PATIENCE:
            print(f"\n🛑 Early stopping at epoch {epoch+1}")
            break

print("\n" + "="*70)
print("✅ 学習完了")
print("="*70)

In [12]:
# 学習履歴
history = {
    'train_loss': [],
    'val_loss': [],
    'val_mae': []
}

best_val_loss = float('inf')
patience_counter = 0

print("="*70)
print("学習開始")
print("="*70)

# 学習ループ
for epoch in range(NUM_EPOCHS):
    print(f"\n{'='*70}")
    print(f"Epoch {epoch+1}/{NUM_EPOCHS}")
    print(f"{'='*70}")

    # Training
    model.train()
    train_loss = 0

    for batch in tqdm(train_loader, desc="Training"):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()

        predictions = model(input_ids, attention_mask)
        loss = criterion(predictions, labels)

        loss.backward()
        optimizer.step()
        scheduler.step()

        train_loss += loss.item()

    avg_train_loss = train_loss / len(train_loader)
    history['train_loss'].append(avg_train_loss)

    # Validation
    val_predictions, val_labels, val_loss = evaluate_model(model, val_loader, device)
    val_mae = mean_absolute_error(val_labels, val_predictions)

    history['val_loss'].append(val_loss)
    history['val_mae'].append(val_mae)

    print(f"\n📊 Results:")
    print(f"  Train Loss: {avg_train_loss:.4f}")
    print(f"  Val Loss: {val_loss:.4f}")
    print(f"  Val MAE: {val_mae:.4f}")

    # Early Stopping & Checkpoint
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
        # モデル保存
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'val_loss': val_loss,
        }, f"{CHECKPOINT_DIR}/best_model.pt")
        print("  ✅ Best model saved!")
    else:
        patience_counter += 1
        print(f"  ⚠️  Patience: {patience_counter}/{EARLY_STOPPING_PATIENCE}")
        if patience_counter >= EARLY_STOPPING_PATIENCE:
            print(f"\n🛑 Early stopping at epoch {epoch+1}")
            break

print("\n" + "="*70)
print("✅ 学習完了")
print("="*70)

学習開始

Epoch 1/20


Training:   0%|          | 0/47 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 384.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 246.12 MiB is free. Process 2468 has 14.50 GiB memory in use. Of the allocated memory 14.33 GiB is allocated by PyTorch, and 42.22 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

## 7. テストセット評価

In [ ]:
# ベストモデルのロード
print("="*70)
print("ベストモデルをロード中...")
print("="*70)

checkpoint = torch.load(f"{CHECKPOINT_DIR}/best_model.pt")
model.load_state_dict(checkpoint['model_state_dict'])
print(f"✅ Best model loaded! (Epoch {checkpoint['epoch']+1}, Val Loss: {checkpoint['val_loss']:.4f})")

# テストセット評価
print("\n" + "="*70)
print("テストセット評価中...")
print("="*70)

test_predictions, test_labels, test_loss = evaluate_model(model, test_loader, device)
print(f"\n📊 Test Loss: {test_loss:.4f}")
print(f"📊 Test MAE: {mean_absolute_error(test_labels, test_predictions):.4f}")

In [ ]:
# 詳細評価関数
def compute_metrics(predictions, labels, trait_names=BIG_FIVE_TRAITS):
    """
    回帰指標と分類指標の両方を計算
    """
    results = {}

    # 中央値を計算（分類用）
    medians = np.median(labels, axis=0)

    print("\n" + "="*70)
    print("Personality Recognition Results")
    print("="*70)

    # 回帰指標
    print("\nRegression Metrics:")
    print("-"*70)
    print(f"{'Trait':<18} {'MAE':>8} {'RMSE':>8} {'Pearson':>10} {'Spearman':>10}")
    print("-"*70)

    regression_metrics = []
    for i, trait in enumerate(trait_names):
        pred = predictions[:, i]
        true = labels[:, i]

        mae = mean_absolute_error(true, pred)
        rmse = np.sqrt(mean_squared_error(true, pred))

        # 相関係数（p値も計算）
        pearson_corr, pearson_p = pearsonr(true, pred)
        spearman_corr, spearman_p = spearmanr(true, pred)

        regression_metrics.append({
            'trait': trait,
            'mae': mae,
            'rmse': rmse,
            'pearson': pearson_corr,
            'spearman': spearman_corr
        })

        print(f"{trait:<18} {mae:>8.4f} {rmse:>8.4f} {pearson_corr:>10.3f} {spearman_corr:>10.3f}")

    # 平均
    avg_mae = np.mean([m['mae'] for m in regression_metrics])
    avg_rmse = np.mean([m['rmse'] for m in regression_metrics])
    avg_pearson = np.mean([m['pearson'] for m in regression_metrics])
    avg_spearman = np.mean([m['spearman'] for m in regression_metrics])

    print("-"*70)
    print(f"{'Average':<18} {avg_mae:>8.4f} {avg_rmse:>8.4f} {avg_pearson:>10.3f} {avg_spearman:>10.3f}")

    # 分類指標（中央値で2値化）
    print("\n\nClassification Metrics (High/Low by median):")
    print("-"*70)
    print(f"{'Trait':<18} {'Acc':>6} {'Bal Acc':>8} {'Prec':>6} {'Rec':>6} {'F1':>6}")
    print("-"*70)

    classification_metrics = []
    for i, trait in enumerate(trait_names):
        pred = predictions[:, i]
        true = labels[:, i]
        median = medians[i]

        # 2値化
        pred_binary = (pred > median).astype(int)
        true_binary = (true > median).astype(int)

        acc = accuracy_score(true_binary, pred_binary)
        bal_acc = balanced_accuracy_score(true_binary, pred_binary)
        prec, rec, f1, _ = precision_recall_fscore_support(
            true_binary, pred_binary, average='binary', zero_division=0
        )

        classification_metrics.append({
            'trait': trait,
            'accuracy': acc,
            'balanced_accuracy': bal_acc,
            'precision': prec,
            'recall': rec,
            'f1': f1
        })

        print(f"{trait:<18} {acc:>6.1%} {bal_acc:>8.1%} {prec:>6.2f} {rec:>6.2f} {f1:>6.2f}")

    # 平均
    avg_acc = np.mean([m['accuracy'] for m in classification_metrics])
    avg_bal_acc = np.mean([m['balanced_accuracy'] for m in classification_metrics])
    avg_prec = np.mean([m['precision'] for m in classification_metrics])
    avg_rec = np.mean([m['recall'] for m in classification_metrics])
    avg_f1 = np.mean([m['f1'] for m in classification_metrics])

    print("-"*70)
    print(f"{'Average':<18} {avg_acc:>6.1%} {avg_bal_acc:>8.1%} {avg_prec:>6.2f} {avg_rec:>6.2f} {avg_f1:>6.2f}")

    # 論文との比較
    print("\n\nComparison with Paper (Fu et al. 2024):")
    print("-"*70)
    print(f"Paper Balanced Accuracy (Monologue): 60.4%")
    print(f"Our Balanced Accuracy:                {avg_bal_acc:.1%}")
    print("="*70)

    results['regression'] = regression_metrics
    results['classification'] = classification_metrics
    results['averages'] = {
        'mae': avg_mae,
        'rmse': avg_rmse,
        'pearson': avg_pearson,
        'spearman': avg_spearman,
        'accuracy': avg_acc,
        'balanced_accuracy': avg_bal_acc
    }

    return results

# 評価実行
results = compute_metrics(test_predictions, test_labels)

## 8. 可視化

In [ ]:
# 学習曲線の可視化
def plot_training_history(history):
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))

    # Loss
    axes[0].plot(history['train_loss'], label='Train Loss', marker='o')
    axes[0].plot(history['val_loss'], label='Val Loss', marker='s')
    axes[0].set_xlabel('Epoch')
    axes[0].set_ylabel('Loss (MAE)')
    axes[0].set_title('Training and Validation Loss')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)

    # MAE
    axes[1].plot(history['val_mae'], label='Val MAE', marker='o', color='green')
    axes[1].set_xlabel('Epoch')
    axes[1].set_ylabel('MAE')
    axes[1].set_title('Validation MAE')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig(f'{CHECKPOINT_DIR}/training_history.png', dpi=300, bbox_inches='tight')
    plt.show()

# 学習曲線をプロット
plot_training_history(history)

In [ ]:
# 散布図: 予測 vs 真値
def plot_predictions(predictions, labels, trait_names=BIG_FIVE_TRAITS):
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    axes = axes.flatten()

    for i, trait in enumerate(trait_names):
        ax = axes[i]

        pred = predictions[:, i]
        true = labels[:, i]

        # 散布図
        ax.scatter(true, pred, alpha=0.5, s=20)

        # 理想線
        min_val = min(true.min(), pred.min())
        max_val = max(true.max(), pred.max())
        ax.plot([min_val, max_val], [min_val, max_val], 'r--', lw=2, label='Ideal')

        # 相関係数
        pearson, _ = pearsonr(true, pred)
        mae = mean_absolute_error(true, pred)

        ax.set_xlabel('True Score')
        ax.set_ylabel('Predicted Score')
        ax.set_title(f'{trait}\nPearson: {pearson:.3f}, MAE: {mae:.3f}')
        ax.legend()
        ax.grid(True, alpha=0.3)

    # 最後のaxesを非表示
    axes[-1].axis('off')

    plt.tight_layout()
    plt.savefig(f'{CHECKPOINT_DIR}/predictions_scatter.png', dpi=300, bbox_inches='tight')
    plt.show()

# 散布図をプロット
plot_predictions(test_predictions, test_labels)

In [ ]:
# 混同行列（分類として評価）
from sklearn.metrics import confusion_matrix

def plot_confusion_matrices(predictions, labels, trait_names=BIG_FIVE_TRAITS):
    medians = np.median(labels, axis=0)

    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    axes = axes.flatten()

    for i, trait in enumerate(trait_names):
        ax = axes[i]

        pred = predictions[:, i]
        true = labels[:, i]
        median = medians[i]

        # 2値化
        pred_binary = (pred > median).astype(int)
        true_binary = (true > median).astype(int)

        # 混同行列
        cm = confusion_matrix(true_binary, pred_binary)

        # ヒートマップ
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax,
                    xticklabels=['Low', 'High'],
                    yticklabels=['Low', 'High'])
        ax.set_xlabel('Predicted')
        ax.set_ylabel('True')
        ax.set_title(f'{trait} - Confusion Matrix')

    # 最後のaxesを非表示
    axes[-1].axis('off')

    plt.tight_layout()
    plt.savefig(f'{CHECKPOINT_DIR}/confusion_matrices.png', dpi=300, bbox_inches='tight')
    plt.show()

# 混同行列をプロット
plot_confusion_matrices(test_predictions, test_labels)

## 9. まとめ

このノートブックでは以下を実装しました：

1. ✅ RealPersonaChatデータセットの読み込みと前処理
2. ✅ LUKE回帰モデルの構築
3. ✅ モノローグ設定での学習
4. ✅ 回帰指標（MAE, RMSE, 相関）と分類指標（Acc, F1）の評価
5. ✅ 論文（Fu et al. 2024）との比較

**次のステップ:**
- データセットの実際のパスを指定して実行
- 学習を実行し、結果を分析
- 発表資料の作成